<a href="https://colab.research.google.com/github/Neetagrg/Mini-Project_2_Spring-2025-WEB-DATA-MINING-CUS-635/blob/main/mini_project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mini Project **2**: Neeta Kumari & Bir Bahadur Gharti Class CUS635

# Install Required Libraries

In [ ]:

!pip install boto3 requests pandas

# Import Required Libraries

In [ ]:
# Import the boto3 library
import boto3
from botocore.config import Config
from botocore import UNSIGNED

# Team 6 folder
TEAM = "TEAM_6/"

# Define the provided S3 bucket name
BUCKET_NAME = "cus635-spring2025"

# Create an anonymous S3 client
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

# Upload the CSV to S3 Download a File from S3

In [ ]:
import boto3
from botocore.config import Config
from botocore import UNSIGNED

# Team
TEAM = "TEAM_6/"
BUCKET_NAME = "cus635-spring2025"

# Create an anonymous S3 client
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

# Download a file from S3
object_name = "TEAM_6/Financial_Articles.csv"  # Corrected file path
new_name = "downloaded_Finance_Articles.csv"

s3.download_file(BUCKET_NAME, object_name, new_name)
print(f"File '{object_name}' downloaded successfully as '{new_name}'!")


File 'TEAM_6/Financial_Articles.csv' downloaded successfully as 'downloaded_Finance_Articles.csv'!


# Pinecone Setup

In [ ]:
!pip install pinecone


In [ ]:
from pinecone import Pinecone

# Initialize Pinecone instance with your API key and environment
pc = Pinecone(
    api_key="pcsk_6akU8Z_2BXXXDSBKbvFCn4sciNM2FeJC6PwAt6wFwQeQjoJKDSjysRbtyBAdUfRv6z87e6",
    environment="us-east-1"
)

# Index name you're trying to access
index_name = "cus635"

# Check if the index exists
if index_name not in pc.list_indexes().names():
    print(f"Index '{index_name}' does not exist.")
else:
    print(f"Successfully connected to index '{index_name}'.")

    # Access your index
    index = pc.Index(index_name)


Successfully connected to index 'cus635'.


# Create the DocumentProcessor Class

In [ ]:
class DocumentProcessor:
    def __init__(self, team, category):
        self.team = team
        self.category = category

    def fetch_documents(self, bucket, prefix):
        response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
        return [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].endswith('.json')]

    def process_and_upload(self, bucket, file_key):
        obj = s3.get_object(Bucket=bucket, Key=file_key)
        content = json.loads(obj['Body'].read().decode('utf-8'))

        # Assume JSON contains at least 'title', 'text', 'published'
        vector = {
            "id": file_key,  # or generate a unique ID
            "values": None,  # Leave this None to use automatic embedding
            "metadata": {
                "title": content.get('title', 'No Title'),
                "text": content.get('text', ''),
                "published": content.get('published', ''),
                "team": self.team,
                "category": self.category
            }
        }
        index.upsert(vectors=[vector])
        print(f"Uploaded: {file_key}")


# Embed Your Query Using llama-text-embed-v2

In [ ]:
pip install -q sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 26.8 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer

# Load a lightweight embedding model
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# Embed the query
query_text = "investment opportunities in stock market"
query_vector = embed_model.encode(query_text).tolist()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

TL;DR Fix for Embedding Pinecone index expects 1024-dimensional vectors

In [ ]:
pip install sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer

# Load the model that outputs 1024-d vectors
model = SentenceTransformer("intfloat/e5-large-v2")

# Embed your query
query_text = "investment opportunities in stock market"
query_vector = model.encode(query_text).tolist()


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

# Perform Pinecone Query with the Embedded Vector

In [ ]:
# Now query with the embedded vector
result = index.query(
    vector=query_vector,
    top_k=5,
    include_metadata=True,
    filter={"category": "Finance"}
)

# Show results
for match in result.get('matches', []):
    print("📰 Title:", match['metadata'].get('title', 'N/A'))
    print("📅 Published:", match['metadata'].get('published', 'N/A'))
    print("📈 Score:", match['score'])
    print("📖 Snippet:", match['metadata'].get('text', '')[:200], "...\n")


📰 Title: Minus 20 Prozent in einer Woche: Kurse von Bitcoin & Co. regelrecht abgestürzt
📅 Published: N/A
📈 Score: -0.00154348754
📖 Snippet: Title: Minus 20 Prozent in einer Woche: Kurse von Bitcoin & Co. regelrecht abgestürzt
Description: Noch vor etwas über einem Monat kostete ein Bitcoin etwa 110.000 US-Dollar. Nach einer dramatischen W ...

📰 Title: Zentralafrikanische Republik führt Meme-Coin ein
📅 Published: N/A
📈 Score: -0.00154348754
📖 Snippet: Title: Zentralafrikanische Republik führt Meme-Coin ein
Description: Der Bitcoin wurde in dem afrikanischen Land als offizielles Zahlungsmittel wieder abgeschafft. Nun soll ein Meme-Coin "die national ...

📰 Title: Bösartiger Code in 200 GitHub-Repositories stiehlt knapp 500.000 Euro
📅 Published: N/A
📈 Score: -0.00154348754
📖 Snippet: Title: Bösartiger Code in 200 GitHub-Repositories stiehlt knapp 500.000 Euro
Description: Eine Malware-Kampagne in GitHub-Repositories hat es auf Bankdaten und Bitcoin-Wallets abgesehen. Der Schadcode ...

📰

# Generating vectors using sentence-transformers/all-MiniLM-L6-v2


In [ ]:
from pinecone import Pinecone
from transformers import AutoTokenizer, AutoModel
import torch
import uuid  # For generating unique IDs

# Initialize Pinecone with the API key using the Pinecone class
pc = Pinecone(api_key="pcsk_6akU8Z_2BXXXDSBKbvFCn4sciNM2FeJC6PwAt6wFwQeQjoJKDSjysRbtyBAdUfRv6z87e6", environment="us-east-1-aws")

# Connect to the Pinecone index
index = pc.Index("cus635")

# Load the model and tokenizer
model_name = "NousResearch/llama-text-embed-v2"  # Llama model for embedding
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Function to embed text
def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings[0].numpy()

# Example documents (replace with your actual articles)
documents = [
    {"id": str(uuid.uuid4()), "text": "Stock market investments offer various opportunities...", "metadata": {"title": "Stock Market Investments", "published": "2025-04-14"}},
    {"id": str(uuid.uuid4()), "text": "Investing in bonds is another way to build wealth...", "metadata": {"title": "Investing in Bonds", "published": "2025-04-12"}},
]

# Generate embeddings and prepare for Pinecone
vectors = [
    (doc["id"], embed_text(doc["text"]).tolist(), doc["metadata"])  # Embed text and prepare vector
    for doc in documents
]

# Ingest documents into Pinecone
index.upsert(vectors=vectors)

print("Documents successfully uploaded to Pinecone.")


OSError: NousResearch/llama-text-embed-v2 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

# Updated Code for Using sentence-transformers/all-MiniLM-L6-v2

In [ ]:
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
import torch
import uuid  # For generating unique IDs

# Initialize Pinecone with the API key using the Pinecone class
pc = Pinecone(api_key="pcsk_6akU8Z_2BXXXDSBKbvFCn4sciNM2FeJC6PwAt6wFwQeQjoJKDSjysRbtyBAdUfRv6z87e6", environment="us-east-1-aws")

# Connect to the Pinecone index
index = pc.Index("cus635")

# Load the SentenceTransformer model (MiniLM for embeddings)
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Function to embed text
def embed_text(text):
    return model.encode(text).tolist()

# Example documents (replace with your actual articles)
documents = [
    {"id": str(uuid.uuid4()), "text": "Stock market investments offer various opportunities...", "metadata": {"title": "Stock Market Investments", "published": "2025-04-14"}},
    {"id": str(uuid.uuid4()), "text": "Investing in bonds is another way to build wealth...", "metadata": {"title": "Investing in Bonds", "published": "2025-04-12"}},
]

# Generate embeddings and prepare for Pinecone
vectors = [
    (doc["id"], embed_text(doc["text"]), doc["metadata"])  # Embed text and prepare vector
    for doc in documents
]

# Ingest documents into Pinecone
index.upsert(vectors=vectors)

print("Documents successfully uploaded to Pinecone.")


PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 14 Apr 2025 22:05:53 GMT', 'Content-Type': 'application/json', 'Content-Length': '103', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '109', 'x-pinecone-request-id': '1986134980668577671', 'x-envoy-upstream-service-time': '61', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Vector dimension 384 does not match the dimension of the index 1024","details":[]}


# Validate Uploads with a Sample Query

In [ ]:
# 1. Import
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer

# 2. Connect to Pinecone
pc = Pinecone(api_key="pcsk_6akU8Z_2BXXXDSBKbvFCn4sciNM2FeJC6PwAt6wFwQeQjoJKDSjysRbtyBAdUfRv6z87e6")
index = pc.Index("cus635")

# 3. Load embedding model (free & local)
embedder = SentenceTransformer('all-MiniLM-L6-v2')
query = "investment opportunities in stock market"
query_vector = embedder.encode(query).tolist()

# 4. Query the index
result = index.query(
    vector=query_vector,
    top_k=5,
    include_metadata=True
)

# 5. Display results
for match in result.get('matches', []):
    print("📰 Title:", match['metadata'].get('title', 'N/A'))
    print("📅 Published:", match['metadata'].get('published', 'N/A'))
    print("📈 Score:", match['score'])
    print("📖 Snippet:", match['metadata'].get('text', '')[:200], "...\n")


PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 14 Apr 2025 21:46:23 GMT', 'Content-Type': 'application/json', 'Content-Length': '103', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '1', 'x-pinecone-request-id': '8419065789753078566', 'x-envoy-upstream-service-time': '1', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Vector dimension 384 does not match the dimension of the index 1024","details":[]}
